In [1]:
import copy
import glob
import h5py
import itertools
import numpy as np
import os
import pandas as pd
import scipy
import scipy.interpolate
import tqdm

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import palettable

In [3]:
import yt
import trident
import unyt as u

In [4]:
import kalepy as kale
import verdict

In [5]:
import one_zone

# Parameters

In [6]:
# Analysis parameters
seed = 15482
np.random.seed( seed )
rng = np.random.default_rng( seed )
load_existing_sightlines = True
verbose = False

In [7]:
# Data management parameters
extracted_sightlines_dir = './data/Mandelker2020'
data_dir = './data/synthetic_data/sample2'
observer_data_dir = './data/synthetic_data_samples/sample2'

In [8]:
# Ray parameters
redshift = 0.25

In [9]:
# Spectra parameters
ions = [
    'H I',
    'O I',
    'C II',
    'C III',
    'C IV',
    'N II',
    'N III',
    'Si II',
    'Si III',
    'Si IV',
    'N V',
    'O VI',
    'Mg II'
]
fields = [
    'H_p0_number_density', 
    'O_p0_number_density',
    'C_p1_number_density',
    'C_p2_number_density',
    'C_p3_number_density',
    'N_p1_number_density',
    'N_p2_number_density',
    'Si_p1_number_density',
    'Si_p2_number_density',
    'Si_p3_number_density',
    'N_p4_number_density',
    'O_p5_number_density',
    'Mg_p1_number_density'
]
snr = 30

# Setup Spectrum Generator and Line Database

In [10]:
# Objects for use
ldb = trident.LineDatabase('lines.txt')

In [11]:
# Add Mg II lines
ldb.add_line( 'Mg', 'II', 2796, use_linetools=True)
ldb.add_line( 'Mg', 'II', 2803, use_linetools=True)

read_sets: Using set file -- 
  /Users/zhafen/repos/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [12]:
# Make sure we have the lower part of this Si IV doublet
ldb.add_line( 'Si', 'IV', 1394, use_linetools=True)

/Users/zhafen/repos/trident/trident/line_database.py:215: UserWarning: Line SiIV 1394 not found in linetools database. Checking adjacent wavelengths.
  warnings.warn( 'Line ' + linetools_id(wavelength) + \


In [13]:
sg_cos = trident.SpectrumGenerator('COS-G130M', line_database=ldb )
sg_cos_160 = trident.SpectrumGenerator('COS-G160M', line_database=ldb )

yt : [INFO     ] 2021-08-02 17:39:22,439 Setting instrument to COS-G130M
yt : [INFO     ] 2021-08-02 17:39:22,442 Setting instrument to COS-G160M


In [14]:
# Spectrum Generator for Mg II from ground
lambda_mg = ldb.select_lines( 'Mg', 'II', 2796 )[0].wavelength * ( 1. + redshift )
sg_mg = trident.SpectrumGenerator(
    lambda_min = lambda_mg - 30.,
    lambda_max = lambda_mg + 30.,
    dlambda = 0.01,
    lsf_kernel = os.path.join( trident.path, 'data', 'lsf_kernels', 'avg_COS.txt' ),
    line_database = ldb,
)

yt : [INFO     ] 2021-08-02 17:39:22,455 Setting instrument to Custom


In [15]:
spectrum_sg_tags = [ '_G130', '_G160', '_MgII']

In [16]:
def plot_ion( sg, element, ion_state, width=6. ):
    '''Save a plot of a particular part of the spectrum for inspection.'''
    
    lines = ldb.select_lines( element, ion_state )
    
    wavelengths = np.array([ _.wavelength for _ in lines ])
    adjusted_wavelengths = wavelengths * ( 1 + redshift )
    
    data_subdir = '{}/ion_spectra/{}{}'.format( data_dir, element, ion_state, )
    os.makedirs( data_subdir, exist_ok=True )
    for k, lambda_a in enumerate( adjusted_wavelengths ):
        if lambda_a - width/2. < sg.lambda_min or lambda_a + width/2 > sg.lambda_max:
            continue
            
        sg.plot_spectrum(
            '{}/spectrum_{:.1f}_sl{:04d}.png'.format( data_subdir, lambda_a, i ),
            lambda_limits = [ lambda_a - width/2, lambda_a + width/2 ]
        )

In [17]:
# Waaaay too much output otherwise
# yt.utilities.logger.disable_stream_logging()

# Load and Format Data

Units are [ g/cm^3, mass fraction, logK, log cm^-2 ] respectively for ['Density', 'Metallicity', 'Temperature', 'hneutralssh'].

In [18]:
i = 1

In [19]:
sightline_fps = glob.glob( os.path.join( extracted_sightlines_dir, '*', '*' ) )

In [20]:
sightline_fp = sightline_fps[i]

In [21]:
# Load data
columns = [ 'x', 'density', 'temperature', 'metallicity', 'velocity_los' ]
units = [ 'kpc', 'g/cm**3', 'K', 'Zsun', 'km/s' ]
ray_df = pd.read_csv( sightline_fp, sep='\s+', names=columns )

# Convert metallicity to Zsun
ray_df['metallicity'] /= 0.02

In [22]:
# Turn into a dictionary for creating a yt dataset
ray_dict = {}
for j, key in enumerate( columns ):
    
    # We add this separately
    if key == 'x':
        continue
    
    ray_dict[key] = yt.YTArray( ray_df[key].values, units[j] )

In [23]:
# Get sightline location in space
locstring = os.path.split( os.path.split( sightline_fp )[0] )[1]
_, x1, y1, z1, _, x2, y2, z2 = locstring.split( '_' )
start = np.array([ x1, y1, z1 ]).astype( float )
end = np.array([ x2, y2, z2 ]).astype( float )

In [24]:
# Add sightline dl parameters
dl = ray_df['x'][1] - ray_df['x'][0]
ray_dict['dl'] = dl
length_code_units = np.linalg.norm( end - start )
length_kpc = ray_df['x'].values[-1] - ray_df['x'].values[0]
position_code_units_to_kpc = length_kpc / length_code_units

In [25]:
# Add location parameters
for j, key in enumerate( [ 'x', 'y', 'z' ]):
    ray_dict[key] = yt.YTArray(
        np.linspace( start[j], end[j], ray_df['x'].size ) * position_code_units_to_kpc,
        'kpc',
    )
    ray_dict['d' + key] = ray_dict[key][1] - ray_dict[key][0]
    
# ray_dict['x'] = ( float( x2 ) - float( x1 ) ) / 2.
# ray_dict['y'] = ( float( y2 ) - float( y1 ) ) / 2.
# ray_dict['z'] = ( float( z2 ) - float( z1 ) ) / 2.

In [26]:
# Add redshift parameters
ray_dict['redshift'] = np.full( ray_dict['density'].shape, redshift )
z_vel = np.sqrt( ( 1 + ray_dict['velocity_los'] / u.c) / ( 1 - ray_dict['velocity_los'] / u.c) ) - 1.
ray_dict['redshift_eff'] = ( 1. + redshift )*( 1. + z_vel ) - 1.

In [27]:
# Other needed terms
extra_attrs = {"data_type": "yt_light_ray", "dimensionality": 3}
field_types = dict([(field, "grid") for field in ray_dict.keys()])

In [28]:
# Format dataset dict
ds = {
    "current_time": 0.,
    "current_redshift": redshift,
    "cosmological_simulation": 0.,
    "domain_left_edge": start,
    "domain_right_edge": end,
    "periodicity": [True]*3,
}

In [29]:
# Save as a dataset
ray_filename = os.path.join( data_dir, 'rays', 'ray_{:03d}.h5'.format( i ) )
yt.save_as_dataset(
    ds,
    ray_filename,
    ray_dict,
    field_types = field_types,
    extra_attrs = extra_attrs,
)

yt : [INFO     ] 2021-08-02 17:39:22,644 Saving field data to yt dataset: ./data/synthetic_data/sample2/rays/ray_001.h5.


'./data/synthetic_data/sample2/rays/ray_001.h5'

In [30]:
# Reload
ray = yt.load( ray_filename )

yt : [INFO     ] 2021-08-02 17:39:22,766 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-08-02 17:39:22,766 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2021-08-02 17:39:22,767 Parameters: domain_left_edge          = [0.7537 0.3587 0.25  ]
yt : [INFO     ] 2021-08-02 17:39:22,767 Parameters: domain_right_edge         = [0.9961 0.5869 0.75  ]
yt : [INFO     ] 2021-08-02 17:39:22,768 Parameters: cosmological_simulation   = 0.0


In [31]:
# temporary fix for yt-4.0 ytdata selection issue
ray.domain_left_edge = ray.domain_left_edge.to('code_length')
ray.domain_right_edge = ray.domain_right_edge.to('code_length')

In [32]:
trident.add_ion_fields(ray, ions=ions, line_database=ldb)

yt : [INFO     ] 2021-08-02 17:39:22,791 Allocating for 6.150e+02 particles


In [34]:
for m, sg in enumerate( [ sg_cos, sg_cos_160, sg_mg ]):

    # Make the spectrum
    sg.make_spectrum( ray, lines=ions, store_observables=True )

    # Save
    sg.save_spectrum(
        '{}/spectrum{}_sl{:04d}.h5'.format( observer_data_dir, spectrum_sg_tags[m], i )
    )
    sg.plot_spectrum(
        '{}/spectrum{}_sl{:04d}.png'.format( observer_data_dir, spectrum_sg_tags[m], i )
    )

yt : [INFO     ] 2021-08-02 17:39:24,823 Creating spectrum
yt : [INFO     ] 2021-08-02 17:39:24,855 Not adding line Ly a: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,856 Not adding line Ly b: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,857 Not adding line Ly c: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,858 Not adding line Ly d: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,858 Not adding line Ly e: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,859 Not adding line Ly 6: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,860 Not adding line Ly 7: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,860 Not adding line Ly 8: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,861 Not adding line Ly 9: insufficient column density
yt : [INFO     ] 2021-08-02 17:39:24,862 Not adding line Ly 10: insufficient column density
yt : [INFO     ] 2021-08-02 17